In [1]:
import json
import pandas as pd
from pandas import json_normalize
from collections import defaultdict
from itertools import groupby

In [2]:
with open('telegram.channels.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [3]:
with open('telegram.messages.2024May1stWeek.json', 'r', encoding='utf-8') as file2:
    data2 = json.load(file2)
    # data2 = pd.read_json(file2)

In [4]:
# -----------------------------------------------
# For normalizing post count and average view count
channels = json_normalize(data)
posts = json_normalize(data2)

In [5]:
#For searching post count
group_post = posts.groupby('peer_id.channel_id')
# Searching Post Count
group_post.size().reset_index(name='post_count').sort_values(by='post_count', ascending= False).head(10)

,peer_id.channel_id,post_count
235,1871239872,1353
41,1457688370,1196
55,1517600807,703
173,1746317237,703
271,2130995355,670
72,1543744698,604
236,1875043295,584
167,1735532540,537
75,1546541186,534
27,1353554094,381


In [6]:

#For searching top ten channels with average view count
group_avg = posts.groupby('peer_id.channel_id')['views']
#Searching Average View Count
avg_view_result = group_avg.mean().reset_index(name='average_views').sort_values(by='average_views', ascending=False).head(10)

avg_view_result['channelId'] = avg_view_result['peer_id.channel_id']

avg_view_result
# -----------------------------------------------

,peer_id.channel_id,average_views,channelId
0,1006503122,432990.000000,1006503122
17,1291462171,41919.300000,1291462171
270,2049814454,34922.414286,2049814454
208,1790991084,24637.075000,1790991084
115,1624745649,21126.669421,1624745649
209,1794331545,19909.375000,1794331545
143,1679901242,19841.000000,1679901242
83,1558737942,18187.074257,1558737942
255,1941415260,17623.833333,1941415260
246,1907154014,16743.800000,1907154014


In [7]:

# -----------------------------------------------
# For searching post author
df = json_normalize(data)

#For searching post author
groups = df.groupby('userName')['channelId']
#Searching post author
groups.nunique().reset_index(name='channel_count').query('channel_count > 1')
# -----------------------------------------------

# Searching Total Views
# groups.views.sum().reset_index(name='total_views').sort_values(by='total_views', ascending=False).head(10)

# Searching Message Count
# groups.size().reset_index(name='message_count').sort_values(by='message_count', ascending=False).head(10)

,userName,channel_count
88,NA,3
384,minkansei133,2


In [9]:
channels_subscribers = channels[['channelId','subscriberCount']]

channels_subscribers['subscriberCount'] = pd.to_numeric(channels_subscribers['subscriberCount'], errors='coerce')

# avg_view_result
# channels_subscribers
merged_df = avg_view_result.merge(channels_subscribers, on='channelId' )

merged_df['performance_value'] = merged_df['average_views'] / merged_df['subscriberCount']

merged_df

# avg_view_result

# channels[['channelId','subscriberCount']]

/var/folders/7f/21mcg9vj6dx3mkg_w2rgb4540000gn/T/ipykernel_59253/1871989975.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  channels_subscribers['subscriberCount'] = pd.to_numeric(channels_subscribers['subscriberCount'], errors='coerce')


,peer_id.channel_id,average_views,channelId,subscriberCount,performance_value
0,1006503122,432990.000000,1006503122,1515810.0,0.285649
1,1291462171,41919.300000,1291462171,NaN,NaN
2,2049814454,34922.414286,2049814454,NaN,NaN
3,1790991084,24637.075000,1790991084,NaN,NaN
4,1624745649,21126.669421,1624745649,NaN,NaN
5,1794331545,19909.375000,1794331545,NaN,NaN
6,1679901242,19841.000000,1679901242,NaN,NaN
7,1558737942,18187.074257,1558737942,NaN,NaN
8,1941415260,17623.833333,1941415260,NaN,NaN
9,1907154014,16743.800000,1907154014,NaN,NaN


In [19]:
posts.groupby('post_author').filter(lambda x: x['peer_id.channel_id'].nunique() > 1)['post_author'].unique()

array(['YU ATARASHI', 'Snow🇲🇲', 'M S', 'Ye Yint', 'Anᴀʙᴇllᴇ', 'Ko Htet',
       'hkk', 'စောရဲနိုင်', '- -', 'KKH', 'Kyaw Soe Oo', 'GY', 'htetmyat',
       'Mr Thor Matt', '𝐇𝐚𝐧 𝐍𝘆𝐞𝐢n 𝐎o', 'Domino'], dtype=object)